In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
!unzip /drive/images.zip


unzip:  cannot find or open /drive/images.zip, /drive/images.zip.zip or /drive/images.zip.ZIP.


In [ ]:
!unzip drive/MyDrive/images.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_tickfont.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_outlinewidth.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_len.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_showticklabels.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_tickformat.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_title.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_showtickprefix.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_ypad.py  
  inflating: images/.venv/lib/p

In [ ]:
import os
# tm = train_auto_model()
# folder_list =  tm.list_of_folders()
# folder_list = [i for i in folder_list if in not [".ipynb_checkpoints" , ".idea" ,".venv"]]

folder_list = [i for i in os.listdir("images") if i not in [".ipynb_checkpoints", ".idea", ".venv" ,".DS_Store"]]
folder_list

['metal',
 'brown-glass',
 'clear_glass_juice%2c_perfume%2c_glass',
 'biological',
 'cardboard',
 'green-glass',
 'single_use_plastic_cutlery_spoon%2c_fork%2c_straw%2c_dish%2c_glass',
 'pizza_box_output',
 'news_paper_waste',
 'trash',
 'bottel',
 'battery',
 'zero_shot_object_detection_with_grounding_dino.ipynb',
 'plastic',
 'paper',
 'cardboard_waste',
 'white-glass',
 'aerosol_waste_cans',
 'pomogranade_peel',
 'aluminum_foil_kitchen',
 'clothes',
 'shoes',
 'pizza_box']

In [ ]:
original_name = sample = os.listdir("images")
originale_second = [i.replace("_"," ").replace("%2c" ," ").replace("-"," ") for i in original_name]
print(originale_second)

for i , j in zip(original_name , originale_second):
  i = f"images/{i}"
  j = f"images/{j}"
  os.rename(i ,j )


['metal', '.DS Store', 'brown glass', 'clear glass juice  perfume  glass', '.ipynb checkpoints', '.venv', 'biological', 'cardboard', 'green glass', 'single use plastic cutlery spoon  fork  straw  dish  glass', '.idea', 'pizza box output', 'news paper waste', 'trash', 'bottel', 'battery', 'zero shot object detection with grounding dino.ipynb', 'plastic', 'paper', 'cardboard waste', 'white glass', 'aerosol waste cans', 'pomogranade peel', 'aluminum foil kitchen', 'clothes', 'shoes', 'pizza box']


In [ ]:
import pathlib
import os
from transformers import AutoProcessor, GroundingDinoForObjectDetection
from PIL import Image
import torch
import shutil  # To move images to another folder
import gc  # Garbage collection for memory management
import matplotlib.pyplot as plt
import matplotlib.patches as patches


In [ ]:
class TrainAutoModel:
    def __init__(self):
        self.folders = []
        self.processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
        self.model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base").to('cuda')  # GPU
        self.boxes = []
        self.scores = []

    def list_of_folders(self, images="images/"):
        return [f for f in pathlib.Path(images).iterdir() if f.is_dir()]

    def list_of_files(self, file):
        return [f for f in pathlib.Path(file).iterdir() if f.is_file()]

    def folder_creation(self):
        self.folders = self.list_of_folders()
        for i in self.folders:
            label_name = i.name  # Get folder name as string
            if not os.path.exists(f"trainImages/{label_name}"):
                os.makedirs(f"trainImages/{label_name}/images")
                os.makedirs(f"trainImages/{label_name}/labels")

    def model_train(self, path, text, label_name, threshold=0.35):
        image = Image.open(path).convert('RGB')
        image = image.resize((640, 480))  # Resize to reduce memory
        inputs = self.processor(images=image, text=text, return_tensors="pt").to('cuda')
        outputs = self.model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]]).to('cuda')  # Image size in (H, W) format
        results = self.processor.image_processor.post_process_object_detection(outputs, threshold=threshold, target_sizes=target_sizes)[0]

        detected = False
        img_w, img_h = image.size

        if len(results['scores']) > 0:  # Check if there are any detections
            detected = True
            label_file_content = []  # Store lines for the YOLOv8 format
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                if score >= threshold:
                    box = box.tolist()
                    self.boxes.append(box)
                    self.scores.append(score.tolist())
                    print(f"Detected {label.item()} with confidence {score} at location {box}")

                    # Generate YOLOv8 format coordinates
                    label_file_content.append(self.generate_yolo_format(box, img_w, img_h, label))

            # Move image and save all detections into one label file
            self.move_image_and_save_label(path, label_file_content, label_name)

        # Clean up large variables and run garbage collection after each image is processed
        del image, inputs, outputs, target_sizes, results
        torch.cuda.empty_cache()  # Clear GPU memory
        gc.collect()  # Run garbage collection

        return detected

    def generate_yolo_format(self, box, img_w, img_h, label):
        # Calculate YOLOv8 format (normalized center_x, center_y, width, height)
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2.0 / img_w
        y_center = (y_min + y_max) / 2.0 / img_h
        width = (x_max - x_min) / img_w
        height = (y_max - y_min) / img_h

        # Assuming label class ID is `0` for now, can be changed based on your use case
        class_id = 0  # Change this to appropriate class mapping if needed
        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    def move_image_and_save_label(self, path, label_file_content, label_name):
        # Create destination directories
        dest_dir_images = f"trainImages/{label_name}/images"
        dest_dir_labels = f"trainImages/{label_name}/labels"
        os.makedirs(dest_dir_images, exist_ok=True)
        os.makedirs(dest_dir_labels, exist_ok=True)

        # Move the image
        image_name = pathlib.Path(path).name
        dest_image_path = f"{dest_dir_images}/{image_name}"
        shutil.move(path, dest_image_path)

        # Write all detections to the YOLOv8 label file
        if image_name.split(".")[-1] == "jpg":
            label_file_path = f"{dest_image_path}/{image_name.replace('.jpg','.txt')}"
        elif image_name.split(".")[-1] == "png":
            label_file_path = f"{dest_image_path}/{image_name.replace('.png','.txt')}"
        else:
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpeg','.txt')}"
        with open(label_file_path, 'w') as f:
            for line in label_file_content:
                f.write(line + "\n")

    def display_image_with_bboxes(self, image, bboxes, labels):
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for bbox, label in zip(bboxes, labels):
            x_min, y_min, x_max, y_max = bbox
            rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            ax.text(x_min, y_min, label, color='w', fontsize=10,
                    bbox=dict(facecolor='r', edgecolor='r', alpha=0.8))

        plt.show()

# Example usage:
tm = TrainAutoModel()

# Get list of folders excluding specific ones
folder_list = [i for i in os.listdir("images") if i not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]]

# Loop through the folders and process the images
for folder in folder_list:
    files = [i for i in tm.list_of_files(f"images/{folder}") if i not in [".DS_Store"]]
    print(folder, len(files))

    for file in files:

        detected = tm.model_train(path=str(file), text=f"{folder}.", label_name=folder)
        if detected:
            print(f"{file} moved and labeled successfully.")
        else:
            print(f"No detection for {file}.")
        gc.collect()  # Additional collection after each image processing

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import gc
gc.collect()